In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from QuantumChessGame import * 
from ChessPuzzles import *
from GameToTensor import *
from ChessPuzzles import chess_puzzles

from MCTS import MCTS_Node

import numpy as np
import pandas as pd 

import QChessNN
import MCTS_NN

In [2]:
torch.manual_seed(42)

#Declare a new model
#NNmodel = QChessNN.QChessNN()



# Load the model
#NNmodel = QChessNN.QChessNN()
NNmodel = torch.jit.load('Puzzle#34.pt')
NNmodel.eval()

RecursiveScriptModule(
  original_name=QChessNN
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (conv2): RecursiveScriptModule(original_name=Conv2d)
  (value_conv): RecursiveScriptModule(original_name=Conv2d)
  (value_fc1): RecursiveScriptModule(original_name=Linear)
  (value_fc2): RecursiveScriptModule(original_name=Linear)
)

In [3]:
import pandas as pd
#import mathplotlib.pyplot as plt
#%mathplotlib inline

In [4]:
game = QuantumChessGame()
game.new_game()

gameData = game.get_game_data()
game_tensor = torch.zeros(1,12,8,8)

game_tensor[0] = gameToTensor(gameData, 0)
#print(game_tensor)
y = torch.zeros(12)

In [5]:
class MCTS_AI:
    def __init__(self):
        return

    def find_best_move(self, game, simVar):
        root = MCTS_Node(game)
        gamedata = game.get_game_data()
        bestmove = root.best_action(gamedata.ply, simVar)
        return bestmove

In [6]:
class NetworkMCTS():
    def __init__(self):
            return

    
    def find_best_move(self, game, model, simVar):
        root = MCTS_NN.MCTS_Node(game, model)
        gamedata = game.get_game_data()
        bestmove = root.best_action(gamedata.ply, model, simVar)
        value = root.getConcreteNodeValue()
        return bestmove, value

In [7]:
mcts_nn = NetworkMCTS()

MCTSAI =  MCTS_AI()

In [8]:

def self_play_game(model, moveMax):
    board_data_B = []
    board_data_W = []
    values_B = []
    values_W = []
    game = QuantumChessGame()
    game.new_game({'initial_state_fen':get_puzzle_fen(34),  'max_split_moves':[0,1]});
    movecode = 0;
    while not game.is_game_over():
        gamedata = game.get_game_data()
    
        #best_move, value = mcts_nn.find_best_move(game, model, 30)

        #best_move = MCTSAI.find_best_move(game, 35)
        
        #print(f"Value {value}")
        #print("found best move")

        
        # Record the state, policy, and value
        if (gamedata.ply % 2 == 0):
            best_move, value = mcts_nn.find_best_move(game, model, 30)
            #best_move = input("Enter your move: ")
            #value = input("Enter the value: ")
            board_data_W.append(gamedata)
            values_W.append(value)

        if (gamedata.ply % 2 == 1):
            #best_move, value = mcts_nn.find_best_move(game, model, 30)
            best_move = input("Enter your move: ")
            value = 0
            board_data_B.append(gamedata)
            values_B.append(value)

        if (gamedata.ply == moveMax):
            return board_data_W, board_data_B, values_W, values_B
        
        print(f"player # {gamedata.ply}")
        print(f"move taken {best_move}")
        
        # Apply the move to the board
        board_state, movecode = game.do_move(best_move)
        game.print_board_and_probabilities()
        
    if(movecode == 2):
        return board_data_W, board_data_B, values_W, values_B  # Return +1 for  white win, 0 for draw, -1 for black win

    if(movecode == 3):
        return board_data_W, board_data_B, values_W, values_B  # Return +1 for  white win, 0 for draw, -1 for black win

    if(movecode == 5):
        return board_data_W, board_data_B, values_W, values_B  # Return +1 for  white win, 0 for draw, -1 for black win

In [9]:
optimizer = torch.optim.Adam(NNmodel.parameters(), lr=0.001)

# Training loop


game_tensor = torch.zeros(1,12,8,8)



for loop in range(10):
    for epoch in range(5):
        print(f"starting epoch {epoch + 1}")
        for game in range(2):
            #print(game)
            board_data_w, board_data_b, values_W, values_B = self_play_game(NNmodel, 7)  # Play a game
            

            
            print(f"game {game + 1} finished")
            # Train the model on the collected game data
        """
        for i in range(len(board_data_b)):
            optimizer.zero_grad()

            game_tensor[0] = gameToTensor(board_data_b[i], 0)
            predicted_value = NNmodel(game_tensor)
            true_value = torch.tensor([[values_B[i]]], dtype=torch.float32)

            # Forward pass
            value_loss = F.mse_loss(predicted_value, true_value)
            # Backpropagate and optimize
            value_loss.backward()
            optimizer.step()
            # Compute loss (value loss + policy loss)
        """
        for i in range(len(board_data_w)):
            optimizer.zero_grad()

            game_tensor[0] = gameToTensor(board_data_w[i], 0)
            predicted_value = NNmodel(game_tensor)
            true_value = torch.tensor([[values_W[i]]], dtype=torch.float32)

            # Forward pass
            value_loss = F.mse_loss(predicted_value, true_value)
            # Backpropagate and optimize
            value_loss.backward()
            optimizer.step()
            # Compute loss (value loss + policy loss)

    print(f"epoch {epoch + 1} finished")
    model_scripted = torch.jit.script(NNmodel) # Export to TorchScript
    model_scripted.save('testExport.pt') # Save




starting epoch 1
player # 1
move taken a8^a7b8
 +-------------------------------------------------+
8|   .    50:k   .     .     .     .     .     .   |
7|  50:k 100:P   .     .     .     .     .     .   |
6| 100:P   .   100:N   .     .     .     .     .   |
5|   .     .     .     .     .     .     .     .   |
4|   .     .     .     .     .     .     .     .   |
3|   .     .     .     .     .     .     .     .   |
2|   .     .     .     .     .     .     .   100:K |
1|   .     .     .     .     .     .     .     .   |
 +-------------------------------------------------+
     a     b     c     d     e     f     g     h
player # 2
move taken c6a7.m1
 +-------------------------------------------------+
8|   .    50:k   .     .     .     .     .     .   |
7| 100:N 100:P   .     .     .     .     .     .   |
6| 100:P   .     .     .     .     .     .     .   |
5|   .     .     .     .     .     .     .     .   |
4|   .     .     .     .     .     .     .     .   |
3|   .     .     .     .  

KeyboardInterrupt: Interrupted by user

In [ ]:
game = QuantumChessGame()
game.new_game({'initial_state_fen':get_puzzle_fen(33),  'max_split_moves':[0,1]});
game_tensor[0] = gameToTensor(game.get_game_data(), 0)

output = NNmodel(game_tensor)
print(output)